<a href="https://colab.research.google.com/github/imthegoodboy/LoRa_/blob/main/LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install transformers datasets peft accelerate bitsandbytes


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import Dataset
import torch


In [7]:
model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
# GPT-2 doesn't have a pad token by default, so we add one:
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)


In [8]:
train_data = [
    {
        "prompt": "User: Hi, my WaveGuard device is overheating.\nSupport:",
        "response": "Hello! I’m sorry you’re facing this issue. Please place the device on a hard surface, ensure the vents are not blocked, and restart it. If it still overheats, I can help you file a warranty request."
    },
    {
        "prompt": "User: My WaveGuard app is stuck on loading.\nSupport:",
        "response": "No worries, I can help with that. First, try closing and reopening the app. If the issue persists, clear the app cache or reinstall it. Let me know what you see after that."
    },
    {
        "prompt": "User: Can I use WaveGuard while traveling?\nSupport:",
        "response": "Yes, you can use WaveGuard while traveling as long as you have an internet connection. For best performance, make sure your connection is stable and your device firmware is updated."
    },
]

def format_example(example):
    # We join prompt + response as one text for causal LM training
    return {
        "text": example["prompt"] + " " + example["response"] + tokenizer.eos_token
    }

formatted_data = [format_example(ex) for ex in train_data]
dataset = Dataset.from_list(formatted_data)
